In [ ]:

%run includes/3-network-setup.ipynb

# Networks (Firewalls)
Summary and detail reports on firewalls deployed across the GCP estate.

## Firewalls

In [ ]:

# get firewalls
firewall_queries = [
    f"""
    select 
    name
    ,'{project}' as project
    ,sourceRanges
    ,destinationRanges
    ,sourceServiceAccounts
    ,sourceTags
    ,targetServiceAccounts
    ,targetTags
    ,denied
    ,direction
    ,SPLIT_PART(network, '/', 10) as network
    ,allowed
    ,priority
    ,disabled
    ,logConfig
    ,description
    from google.compute.firewalls where project = '{project}' and name is not null
    """
    for project in networked_projects
]
firewalls_df = run_stackql_queries(firewall_queries)
firewall_projects = list(firewalls_df['project'].unique())
firewalls_df

## Firewalls with Public Access

In [ ]:

# get firewalls with public access
fw_public_access_queries = [
    f"""
    SELECT sourceRange, * FROM
        (
        SELECT  
            name
            ,JSON_EXTRACT(sourceRanges, '$[0]') as sourceRange
            ,destinationRanges
            ,sourceServiceAccounts
            ,sourceTags
            ,targetServiceAccounts
            ,targetTags
            ,direction
            ,SPLIT_PART(network, '/', 10) as network
            ,JSON_EXTRACT(denied, '$[0].ports[0]') as denied_ports
            ,JSON_EXTRACT(denied, '$[0].IPProtocol') as denied_protocols
            ,allowed
            ,priority
            ,disabled
            ,logConfig
            ,description
        from google.compute.firewalls where project = '{project}' and name is not null
        ) as firewalls
    WHERE sourceRange = '0.0.0.0/0'
    """
    for project in firewall_projects
]
fw_public_access_df = run_stackql_queries(fw_public_access_queries)
fw_public_access_df